In [22]:
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display
from numpy import NaN

In [5]:
display(pd.read_csv('C:\\Users\\amisa\\Documents\\Python\\Final_Project.git\\2022_All_Countries_Region_Mobility_Report.csv', nrows=2).head())
display(pd.read_csv('C:\\Users\\amisa\\Documents\\Python\\Final_Project.git\\2022_All_Countries_Region_Mobility_Report.csv', nrows=2).tail())

,country_region_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0


,country_region_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0


In [6]:
disk_engine = create_engine('sqlite:///311_8M.db')

In [4]:
x = pd.read_csv("C:\\Users\\amisa\\Documents\\Python\\Final_Project.git\\ISO 2 to 3.csv")
dx = pd.DataFrame(x, columns=["Alpha-2 code","Alpha-3 code"])
dict = {}
for x in dx.iterrows():
    dict.update({x[1]["Alpha-2 code"].replace("\"", "").replace(" ", ""): x[1]["Alpha-3 code"].replace("\"", "").replace(" ", "")})
    

In [62]:
start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('C:\\Users\\amisa\\Documents\\Python\\Final_Project.git\\2022_All_Countries_Region_Mobility_Report.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['date'] = pd.to_datetime(df['date']) # Convert to datetimes
    df["retail_and_recreation_percent_change_from_baseline"]    = (df["retail_and_recreation_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["grocery_and_pharmacy_percent_change_from_baseline"]     = (df["grocery_and_pharmacy_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["parks_percent_change_from_baseline"]                    = (df["parks_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["transit_stations_percent_change_from_baseline"]         = (df["transit_stations_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["workplaces_percent_change_from_baseline"]               = (df["workplaces_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["residential_percent_change_from_baseline"]              = (df["residential_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["country_region_code"]                                   = df["country_region_code"].map(dict)
    df.index += index_start

    # Remove the un-interesting columns
    columns = ["country_region_code","date","retail_and_recreation_percent_change_from_baseline","grocery_and_pharmacy_percent_change_from_baseline","parks_percent_change_from_baseline","transit_stations_percent_change_from_baseline","workplaces_percent_change_from_baseline","residential_percent_change_from_baseline"]

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1

AttributeError: 'DataFrame' object has no attribute 'datetime'

In [7]:
names = pd.read_sql_query("SELECT DISTINCT(country_region_code) from data", disk_engine)


In [73]:
dt = pd.read_sql_query("SELECT strftime(date(date),) from data LIMIT 10",disk_engine)
dt

OperationalError: (sqlite3.OperationalError) near ")": syntax error
[SQL: SELECT strftime(date(date)/1000,) from data LIMIT 10]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [8]:
contained_names = []
for x in names.iterrows():
    contained_names.append(x[1]["country_region_code"])

In [66]:
Samples = pd.DataFrame()
for x in contained_names:
    i = pd.read_sql_query(f"SELECT country_region_code,date,retail_and_recreation_percent_change_from_baseline from data where country_region_code == \"{x}\" AND date(date,'unixepoch') between date('2020-2-15','unixepoch') and date('2020-12-31','unixepoch') LIMIT 1000", disk_engine)
    Samples = pd.concat([Samples,i])
    print(Samples)
    break

Empty DataFrame
Columns: [country_region_code, date, retail_and_recreation_percent_change_from_baseline]
Index: []


In [45]:
Samples["retail_and_recreation_percent_change_from_baseline"] = Samples["retail_and_recreation_percent_change_from_baseline"].fillna(method = "ffill")

In [64]:
fig = px.scatter_geo(Samples, locations="country_region_code",
                     hover_name="country_region_code", size="retail_and_recreation_percent_change_from_baseline",
                     animation_frame="date",
                     animation_group="country_region_code",
                     projection="natural earth")
fig.show()

ValueError: Value of 'size' is not the name of a column in 'data_frame'. Expected one of [] but received: retail_and_recreation_percent_change_from_baseline